In [63]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
!ls

solution.ipynb       tic-tac-toe.data.txt


In [27]:
df = pd.read_csv("tic-tac-toe.data.txt", sep=',', names = list(range(9)) + ['target'])
df.head()

,0,1,2,3,4,5,6,7,8,target
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [28]:
import copy
def dummy_encode_categorical_columns(data):
    result_data = copy.deepcopy(data)
    for column in data.columns.values:
        result_data = pd.concat(
            [result_data, pd.get_dummies(result_data[column],
                                         prefix = column, prefix_sep = '_')], axis = 1)
        del result_data[column]
    return result_data
df = dummy_encode_categorical_columns(df)
df['target'] = df['target_positive']
df = df.drop(['target_positive', 'target_negative'], axis=1)
df.head()

,0_b,0_o,0_x,1_b,1_o,1_x,2_b,2_o,2_x,3_b,...,6_b,6_o,6_x,7_b,7_o,7_x,8_b,8_o,8_x,target
0,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,1
1,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,0,1,0,1,0,1
2,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,1,0,0,0,1,1
3,0,0,1,0,0,1,0,0,1,0,...,0,1,0,1,0,0,1,0,0,1
4,0,0,1,0,0,1,0,0,1,0,...,1,0,0,0,1,0,1,0,0,1


In [31]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.3)
len(df_train), len(df_test)

(670, 288)

In [35]:
X_train, y_train = df_train.drop(['target'], axis=1), df_train['target']
X_test, y_test = df_test.drop(['target'], axis=1), df_test['target']

In [40]:
X_train_pos = np.array(X_train[y_train == 1])
X_train_neg = np.array(X_train[y_train == 0])

In [74]:
KOEFF_PODGONA = 0.8
def get_impact(obj, X):
    impact = 0
    for x in X:
        common_ids = (obj == x)
        closure_power = np.sum(np.all(obj[common_ids] == X[:, common_ids], axis=1)) - 1
        assert(closure_power >= 0)
        impact += closure_power
    impact /= len(X)
    return impact

y_pred = np.zeros(len(X_test), dtype=np.int32)
for i, test_obj in tqdm(enumerate(np.array(X_test)), position=0):
    positive_impact = get_impact(test_obj, X_train_pos)
    negative_impact = get_impact(test_obj, X_train_neg)
    y_pred[i] = (positive_impact * KOEFF_PODGONA > negative_impact)

288it [00:05, 53.86it/s]


In [75]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("acc : {}, precision : {}, recall : {}".format(accuracy, precision, recall))

acc : 0.7604166666666666, precision : 0.7580645161290323, recall : 0.9543147208121827
